## Analysis of Fashion Text

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data/w-fashion',index_col=0)

df[:3]

,img_link,name,price
0,https://ke.jumia.is/pLMwunB9ALtsfocE238j1DBn4_...,Air Mesh Lady Training Dance Shoes Light Weigh...,KSh 3300
1,https://ke.jumia.is/v-XE6DB1Vo2_uCqmyFAam7k-dN...,Print Off Shoulder Women Dress - Light Green,KSh 700
2,https://ke.jumia.is/4GtkG8rZz81JUvUmOZ6dgu0y0U...,Women Cashmeres Belted Wool Coat - Brown,KSh 1800 - KSh 2200


In [4]:
#### Word Count

In [7]:
df['word_count'] = df['name'].apply(lambda x: len(str(x).split(" ")))
df[['name','word_count']][:3]

,name,word_count
0,Air Mesh Lady Training Dance Shoes Light Weigh...,13
1,Print Off Shoulder Women Dress - Light Green,8
2,Women Cashmeres Belted Wool Coat - Brown,7
